## 1. Raising Exceptions

In [ ]:
# Use 'raise' to throw an exception

def set_age(age):
    if not isinstance(age, int):
        raise TypeError("Age must be an integer")
    if age < 0:
        raise ValueError("Age cannot be negative")
    if age > 150:
        raise ValueError("Age cannot be more than 150")
    return age

# Test
test_values = [25, -5, 200, "thirty", 0]

for val in test_values:
    try:
        result = set_age(val)
        print(f"  ✓ set_age({val}) = {result}")
    except (TypeError, ValueError) as e:
        print(f"  ✗ set_age({val}): {type(e).__name__} - {e}")

In [ ]:
# Re-raising exceptions
def process_data(data):
    try:
        result = data['value'] * 2
        return result
    except KeyError:
        print("Logging: Missing 'value' key")
        raise  # Re-raise the same exception

try:
    process_data({})  # Missing 'value'
except KeyError as e:
    print(f"Caught re-raised exception: {e}")

In [ ]:
# Raise different exception
def divide_numbers(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        raise ValueError(f"Cannot divide {a} by zero") from None

try:
    divide_numbers(10, 0)
except ValueError as e:
    print(f"ValueError: {e}")

## 2. Creating Custom Exceptions

In [ ]:
# Simple custom exception
class InvalidEmailError(Exception):
    """Raised when email format is invalid"""
    pass

def validate_email(email):
    if '@' not in email:
        raise InvalidEmailError(f"'{email}' is not a valid email")
    return True

# Test
emails = ["user@example.com", "invalid-email", "test@domain"]

for email in emails:
    try:
        validate_email(email)
        print(f"  ✓ {email} is valid")
    except InvalidEmailError as e:
        print(f"  ✗ {e}")

In [ ]:
# Custom exception with additional data
class ValidationError(Exception):
    """Exception with field and message"""
    
    def __init__(self, field, message):
        self.field = field
        self.message = message
        super().__init__(f"{field}: {message}")

class User:
    def __init__(self, name, email, age):
        self.set_name(name)
        self.set_email(email)
        self.set_age(age)
    
    def set_name(self, name):
        if not name or len(name) < 2:
            raise ValidationError("name", "Must be at least 2 characters")
        self.name = name
    
    def set_email(self, email):
        if '@' not in email:
            raise ValidationError("email", "Invalid email format")
        self.email = email
    
    def set_age(self, age):
        if not isinstance(age, int) or age < 0:
            raise ValidationError("age", "Must be a positive integer")
        self.age = age

# Test
test_users = [
    ("Alice", "alice@example.com", 25),
    ("A", "bob@example.com", 30),
    ("Charlie", "invalid", 35),
    ("Diana", "diana@example.com", -5)
]

print("Creating users:")
for name, email, age in test_users:
    try:
        user = User(name, email, age)
        print(f"  ✓ Created: {user.name}")
    except ValidationError as e:
        print(f"  ✗ Field '{e.field}': {e.message}")

In [ ]:
# Exception hierarchy for a domain
class BankError(Exception):
    """Base exception for banking operations"""
    pass

class InsufficientFundsError(BankError):
    """Raised when account has insufficient funds"""
    def __init__(self, balance, amount):
        self.balance = balance
        self.amount = amount
        super().__init__(f"Cannot withdraw ${amount:.2f} from balance of ${balance:.2f}")

class AccountLockedError(BankError):
    """Raised when account is locked"""
    def __init__(self, reason="Unknown"):
        self.reason = reason
        super().__init__(f"Account is locked: {reason}")

class InvalidTransactionError(BankError):
    """Raised for invalid transaction"""
    pass

class Account:
    def __init__(self, balance=0):
        self.balance = balance
        self.locked = False
    
    def withdraw(self, amount):
        if self.locked:
            raise AccountLockedError("Too many failed attempts")
        if amount <= 0:
            raise InvalidTransactionError("Amount must be positive")
        if amount > self.balance:
            raise InsufficientFundsError(self.balance, amount)
        self.balance -= amount
        return self.balance

# Test
account = Account(100)

transactions = [50, 30, 50, -10]

print(f"Starting balance: ${account.balance:.2f}")
for amount in transactions:
    try:
        new_balance = account.withdraw(amount)
        print(f"  ✓ Withdrew ${amount:.2f}, balance: ${new_balance:.2f}")
    except BankError as e:
        print(f"  ✗ {type(e).__name__}: {e}")

## 3. Exception Chaining

In [ ]:
# Chain exceptions to preserve context
class DatabaseError(Exception):
    pass

class UserNotFoundError(Exception):
    pass

def get_user_from_db(user_id):
    # Simulate database lookup
    if user_id < 0:
        raise ValueError("Invalid user ID")
    if user_id > 100:
        raise KeyError(f"User {user_id} not found")
    return {"id": user_id, "name": f"User{user_id}"}

def find_user(user_id):
    try:
        return get_user_from_db(user_id)
    except (ValueError, KeyError) as e:
        # Chain the original exception
        raise UserNotFoundError(f"Could not find user {user_id}") from e

# Test
for uid in [50, -5, 150]:
    print(f"\nLooking up user {uid}:")
    try:
        user = find_user(uid)
        print(f"  Found: {user}")
    except UserNotFoundError as e:
        print(f"  Error: {e}")
        print(f"  Caused by: {type(e.__cause__).__name__}: {e.__cause__}")

## 4. Complete Example: Order Processing System

In [ ]:
# Complete exception hierarchy for order system

class OrderError(Exception):
    """Base exception for order operations"""
    pass

class ProductNotFoundError(OrderError):
    def __init__(self, product_id):
        self.product_id = product_id
        super().__init__(f"Product '{product_id}' not found")

class OutOfStockError(OrderError):
    def __init__(self, product, requested, available):
        self.product = product
        self.requested = requested
        self.available = available
        super().__init__(f"'{product}': requested {requested}, only {available} available")

class PaymentError(OrderError):
    pass

class PaymentDeclinedError(PaymentError):
    def __init__(self, reason):
        self.reason = reason
        super().__init__(f"Payment declined: {reason}")

class InvalidOrderError(OrderError):
    def __init__(self, errors):
        self.errors = errors
        super().__init__(f"Invalid order: {', '.join(errors)}")

# Product catalog
PRODUCTS = {
    "LAPTOP": {"name": "Laptop", "price": 999.99, "stock": 5},
    "MOUSE": {"name": "Mouse", "price": 29.99, "stock": 50},
    "KEYBOARD": {"name": "Keyboard", "price": 79.99, "stock": 0},  # Out of stock
}

class Order:
    def __init__(self):
        self.items = []
        self.total = 0
        self.status = "pending"
    
    def add_item(self, product_id, quantity):
        """Add item to order"""
        # Check product exists
        if product_id not in PRODUCTS:
            raise ProductNotFoundError(product_id)
        
        product = PRODUCTS[product_id]
        
        # Check stock
        if product["stock"] < quantity:
            raise OutOfStockError(product["name"], quantity, product["stock"])
        
        # Add to order
        self.items.append({
            "product_id": product_id,
            "name": product["name"],
            "price": product["price"],
            "quantity": quantity
        })
        self.total += product["price"] * quantity
    
    def validate(self):
        """Validate the order"""
        errors = []
        
        if not self.items:
            errors.append("Order is empty")
        
        if self.total <= 0:
            errors.append("Invalid total")
        
        if errors:
            raise InvalidOrderError(errors)
        
        return True
    
    def process_payment(self, card_number):
        """Process payment"""
        # Simulate payment processing
        if card_number.startswith("0000"):
            raise PaymentDeclinedError("Card reported as stolen")
        if not card_number.isdigit() or len(card_number) != 16:
            raise PaymentDeclinedError("Invalid card number")
        
        self.status = "paid"
        return True
    
    def __str__(self):
        items_str = ", ".join(f"{i['name']} x{i['quantity']}" for i in self.items)
        return f"Order({items_str}) = ${self.total:.2f} [{self.status}]"

# Demo function
def place_order(items, card_number):
    """Place a complete order"""
    order = Order()
    
    print(f"\n📦 Processing order...")
    
    try:
        # Add items
        for product_id, qty in items:
            try:
                order.add_item(product_id, qty)
                print(f"  ✓ Added {product_id} x{qty}")
            except (ProductNotFoundError, OutOfStockError) as e:
                print(f"  ✗ {e}")
        
        # Validate
        order.validate()
        print(f"  ✓ Order validated: ${order.total:.2f}")
        
        # Payment
        order.process_payment(card_number)
        print(f"  ✓ Payment successful")
        print(f"  🎉 {order}")
        return order
    
    except InvalidOrderError as e:
        print(f"  ✗ {e}")
    except PaymentDeclinedError as e:
        print(f"  ✗ {e}")
    except OrderError as e:
        print(f"  ✗ Order failed: {e}")
    
    return None

# Test scenarios
print("🛒 ORDER PROCESSING SYSTEM")
print("=" * 50)

# Successful order
place_order([("LAPTOP", 1), ("MOUSE", 2)], "4111111111111111")

# Order with out of stock item
place_order([("LAPTOP", 1), ("KEYBOARD", 1)], "4111111111111111")

# Order with invalid product
place_order([("LAPTOP", 1), ("TABLET", 1)], "4111111111111111")

# Empty order
place_order([], "4111111111111111")

# Declined payment
place_order([("MOUSE", 1)], "0000111111111111")

## Summary

### Raising Exceptions:

```python
raise ValueError("message")    # Raise new exception
raise                          # Re-raise current exception
raise NewError() from e        # Chain exceptions
raise NewError() from None     # Suppress chain
```

### Custom Exception Pattern:

```python
class CustomError(Exception):
    def __init__(self, details):
        self.details = details
        super().__init__(str(details))
```

### Exception Hierarchy Best Practices:

| Level | Example |
|-------|----------|
| Base | `AppError` |
| Domain | `DatabaseError`, `ValidationError` |
| Specific | `UserNotFoundError`, `EmailInvalidError` |

### When to Create Custom Exceptions:
1. Domain-specific errors
2. Need additional error data
3. Group related errors
4. Improve error handling

### Next Lesson: Regular Expressions